# Hello Mr. Malaz

I added `kaggle.json` to my `C:\Users\laila\.kaggle` so the code runs correctly. If you face any issues running the notebook, this might be it.

laila haddad 202010537

## 1 , 2 : Read and Process the Data

Installing the Kaggle API and Downloading the Dataset

In [2]:
%pip install kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c titanic

import zipfile
with zipfile.ZipFile('titanic.zip', 'r') as zip_ref:
    zip_ref.extractall('.')


Loading the Dataset into a DataFrame

In [3]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Handling Missing Values

In [4]:
missing_values = train_df.isnull().sum()
print(missing_values[missing_values > 0])

train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
train_df['Embarked'].fillna(train_df['Embarked'].mode()[0], inplace=True)
train_df['Cabin'].fillna('Unknown', inplace=True)

print(train_df.isnull().sum())

Age         177
Cabin       687
Embarked      2
dtype: int64
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64


C:\Users\laila\AppData\Local\Temp\ipykernel_56772\1370016188.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
C:\Users\laila\AppData\Local\Temp\ipykernel_56772\1370016188.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

Feature Scaling

In [5]:

from sklearn.preprocessing import StandardScaler

features_to_scale = ['Age', 'Fare']
scaler = StandardScaler()
train_df[features_to_scale] = scaler.fit_transform(train_df[features_to_scale])

train_df[features_to_scale].head()


,Age,Fare
0,-0.565736,-0.502445
1,0.663861,0.786845
2,-0.258337,-0.488854
3,0.433312,0.420730
4,0.433312,-0.486337


Removing Duplicate Data

In [6]:
duplicates = train_df.duplicated().sum()
train_df.drop_duplicates(inplace=True)
print(train_df.duplicated().sum())


0


Converting Categorical Data into Numerical Format

In [7]:

train_df_num=train_df.select_dtypes(include=[np.number])
train_df_obj=train_df.select_dtypes(include=[object])

from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_encoder = onehot_encoder.fit(train_df_obj)
train_df_obj = onehot_encoder.transform(train_df_obj)



In [8]:

encoded_df = pd.DataFrame(train_df_obj, index=train_df.index)
train_df = pd.concat([train_df_num, encoded_df], axis=1)
train_df.head()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,0,1,2,...,1715,1716,1717,1718,1719,1720,1721,1722,1723,1724
0,1,0,3,-0.565736,1,0,-0.502445,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,2,1,1,0.663861,1,0,0.786845,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,1,3,-0.258337,0,0,-0.488854,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,4,1,1,0.433312,1,0,0.420730,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,3,0.433312,0,0,-0.486337,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


## 3 - Model Experimentation :  

Importing Necessary Libraries

In [9]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, precision_recall_curve


In [10]:

X = train_df.drop(columns=['Survived'])
y = train_df['Survived']
X.columns = X.columns.astype(str)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### SGDClassifier

In [11]:
sgd_clf = SGDClassifier(random_state=42)

sgd_params = {
    'loss': ['hinge', 'log_loss', 'modified_huber'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [1000, 2000, 3000]
}

sgd_grid = GridSearchCV(sgd_clf, sgd_params, cv=5, scoring='accuracy')
sgd_grid.fit(X_train, y_train)

print(f"Best hyperparameters for SGDClassifier: {sgd_grid.best_params_}")

Best hyperparameters for SGDClassifier: {'alpha': 0.01, 'loss': 'log_loss', 'max_iter': 1000}


In [12]:

y_pred_sgd = sgd_grid.predict(X_test)

precision_sgd = precision_score(y_test, y_pred_sgd)
recall_sgd = recall_score(y_test, y_pred_sgd)

print(f"SGDClassifier Precision: {precision_sgd}")
print(f"SGDClassifier Recall: {recall_sgd}")
y_train_scores = cross_val_predict(sgd_grid.best_estimator_, X_train, y_train, cv=5, method="decision_function")


SGDClassifier Precision: 1.0
SGDClassifier Recall: 0.013513513513513514


### Random Forest Classifier


In [13]:

rf_clf = RandomForestClassifier(random_state=42)

rf_params = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


rf_grid = GridSearchCV(rf_clf, rf_params, cv=5, scoring='accuracy')
rf_grid.fit(X_train, y_train)

print(f"Best hyperparameters for RandomForestClassifier: {rf_grid.best_params_}")


Best hyperparameters for RandomForestClassifier: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}


In [15]:

y_pred_rf = rf_grid.predict(X_test)

precision_rf = precision_score(y_test, y_pred_rf)
recall_rf = recall_score(y_test, y_pred_rf)

print(f"RandomForestClassifier Precision: {precision_rf}")
print(f"RandomForestClassifier Recall: {recall_rf}")


RandomForestClassifier Precision: 0.8461538461538461
RandomForestClassifier Recall: 0.5945945945945946


## 4 - Model Evaluation : 

## Conclusion

### SGDClassifier
**Precision**: 1.0
**Recall**: 0.013

### RandomForestClassifier
**Precision**: 0.84
**Recall**: 0.59

The RandomForestClassifier model did much better, giving the highest precision and recall scores although it took more time to train. it is the best fit because it handles non-inearity well, and it is less likey to overfit
